LOADING DATA

In [ ]:
!pip install tensorflow
!pip install opencv-python

DATA GATHERING

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import multiprocessing
import functools
import matplotlib.pyplot as plt

In [2]:
def init(l):
    global lock
    lock = l

In [3]:
def read_csv(img,data,lbl,rot,id):
    rows = open('dataset4_csv/ID_'+str(id)+'.csv').read().strip().split('\n')
    for row in rows:
        row = row.split(',')
        (x,y,label,rotation,pathimg) = row
        image =  load_img(pathimg, target_size=(256, 256))
        image = img_to_array(image)
        lock.acquire()
        img.append(image)
        data.append([x,y])
        lbl.append(label)
        rot.append(rotation)
        lock.release()

In [4]:
if __name__ == '__main__':
    
    manager = multiprocessing.Manager()
    img = manager.list()
    data = manager.list()
    lbl = manager.list()
    rot = manager.list()
    
    l = multiprocessing.Lock()
    
    df = pd.read_csv('izzaspr.csv')
    
    ID = df['Star ID'].tolist()[:37]

    partial_read_csv = functools.partial(read_csv, img, data, lbl, rot)

    pool = multiprocessing.Pool(processes=36,initializer=init, initargs=(l,))
    pool.map(partial_read_csv, ID)

In [12]:
pool.close()
pool.terminate()

In [13]:
img = np.array(img,dtype='float32')
data = np.array(data,dtype='float32')
lbl = np.array(lbl)
rot = np.array(rot)

In [ ]:
img.max()

In [ ]:
plt.imshow(img[0])
lbl[0]

In [ ]:
lb = LabelBinarizer()
lbl = lb.fit_transform(lbl)
pd.DataFrame(lbl, columns=lb.classes_)

In [ ]:
ro = LabelBinarizer()
rot = ro.fit_transform(rot)
pd.DataFrame(rot, columns=ro.classes_)

In [16]:
split = train_test_split(img, lbl, rot, data, test_size=0.1)
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainRotations, testRotations) = split[4:6]
(trainCoordinates, testCoordinates) = split[6:8]
split = train_test_split(trainImages, trainLabels, trainRotations, trainCoordinates, test_size=0.1)
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainRotations, valRotations) = split[4:6]
(trainCoordinates, valCoordinates) = split[6:8]

In [ ]:
del img, data, lbl, rot, l, manager, partial_read_csv, pool, init, read_csv, lb, ro, split 

MODEL CREATION

In [21]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Rescaling, Dropout, BatchNormalization, Cropping2D, Lambda
from keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Adam

In [22]:
#input layer
inputs = Input(shape=(256,256,3))
scaled_inputs = Rescaling(1/255)(inputs)

#label branch
# l = Cropping2D(cropping=((58,90),(51,82)))(scaled_inputs)
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.2)(l)
l = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.3)(l)
l = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(l)
l = BatchNormalization()(l)
l = MaxPooling2D()(l)
l = Dropout(0.4)(l)
l = Flatten()(l)
l = Dense(256, activation='relu', kernel_initializer='he_uniform')(l)
l = BatchNormalization()(l)
l = Dropout(0.5)(l)
l = Dense(37, activation = 'softmax', name='label')(l) #change here

#rot branch
# r = Cropping2D(cropping=((58,90),(51,82)))(scaled_inputs)
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(scaled_inputs)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.2)(r)
r = Conv2D(32, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.3)(r)
r = Conv2D(16, (3,3), 1, activation = 'relu', kernel_initializer='he_uniform', padding='same')(r)
r = BatchNormalization()(r)
r = MaxPooling2D()(r)
r = Dropout(0.4)(r)
r = Flatten()(r)
r = Dense(256, activation='relu', kernel_initializer='he_uniform')(r)
r = BatchNormalization()(r)
r = Dropout(0.5)(r)
r = Dense(15, activation = 'softmax', name='rotation')(r)

#ann branch
# a = Cropping2D(cropping=((58,90),(51,82)))(scaled_inputs)
a = Conv2D(16, (3,3), 1, activation = 'relu')(scaled_inputs)
a = MaxPooling2D()(a)
a = Conv2D(32, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(64, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Conv2D(128, (3,3), 1, activation = 'relu')(a)
a = MaxPooling2D()(a)
a = Flatten()(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear', kernel_regularizer='l2', bias_regularizer='l2')(a)
a = Dense(32, activation = 'linear')(a)
a = Dense(2, activation = 'linear', name='coordinate')(a)

#model
model = Model(
	inputs=inputs,
	outputs=(l, r, a))

#compile
model.compile(loss={
	'label': 'categorical_crossentropy',
	'rotation': 'categorical_crossentropy',
	'coordinate': 'mse',
}, optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
del inputs, scaled_inputs, l, r, a

TRAINING

In [25]:
from matplotlib import pyplot as plt

In [ ]:
hist = model.fit(
	trainImages, {
	'label': trainLabels,
	'rotation': trainRotations,
	'coordinate': trainCoordinates,
},
	validation_data= (valImages, {
	'label': valLabels,
	'rotation': valRotations,
	'coordinate': valCoordinates,
}),
	batch_size=64,
	epochs=40)

model.save('superz.h5')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('lossz.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['label_accuracy'], color='teal', label='label_accuracy')
plt.plot(hist.history['val_label_accuracy'], color='orange', label='val_label_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('labelzacc.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['rotation_accuracy'], color='teal', label='rotation_accuracy')
plt.plot(hist.history['val_rotation_accuracy'], color='orange', label='val_rotation_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('rotationzacc.PNG')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['coordinate_accuracy'], color='teal', label='coordinate_accuracy')
plt.plot(hist.history['val_coordinate_accuracy'], color='orange', label='val_coorrdinate_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc='upper left')
plt.savefig('coordinatezacc.PNG')

TRAINING FROM H5

In [ ]:
# import numpy as np
# import cv2
# from keras.models import load_model
# from matplotlib import pyplot as plt
# from keras.utils.vis_utils import plot_model

In [ ]:
# model = load_model('supery.h5')

# # model.compile(loss={
# # 	'label': 'categorical_crossentropy',
# # 	'rotation': 'categorical_crossentropy',
# # 	'coordinate': 'mse',
# # }, optimizer='adam', metrics=['accuracy'])
# plot_model(model)

In [ ]:
# hist = model.fit(
# 	trainImages, {
# 	'label': trainLabels,
# 	'rotation': trainRotations,
# 	'coordinate': trainCoordinates,
# },
# 	validation_data= (valImages, {
# 	'label': valLabels,
# 	'rotation': valRotations,
# 	'coordinate': valCoordinates,
# }),
# 	batch_size=32,
# 	epochs=20)

In [ ]:
# fig = plt.figure()
# plt.plot(hist.history['loss'], color='teal', label='loss')
# plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
# fig.suptitle('Loss', fontsize=20)
# plt.legend(loc='upper left')
# plt.savefig('lossx.PNG')

In [ ]:
# fig = plt.figure()
# plt.plot(hist.history['label_accuracy'], color='teal', label='label_accuracy')
# plt.plot(hist.history['val_label_accuracy'], color='orange', label='val_label_accuracy')
# fig.suptitle('Accuracy', fontsize=20)
# plt.legend(loc='upper left')
# plt.savefig('labelxacc.PNG')

In [ ]:
# fig = plt.figure()
# plt.plot(hist.history['rotation_accuracy'], color='teal', label='rotation_accuracy')
# plt.plot(hist.history['val_rotation_accuracy'], color='orange', label='val_rotation_accuracy')
# fig.suptitle('Accuracy', fontsize=20)
# plt.legend(loc='upper left')
# plt.savefig('rotationxacc.PNG')

In [ ]:
# fig = plt.figure()
# plt.plot(hist.history['coordinate_accuracy'], color='teal', label='coordinate_accuracy')
# plt.plot(hist.history['val_coordinate_accuracy'], color='orange', label='val_coorrdinate_accuracy')
# fig.suptitle('Accuracy', fontsize=20)
# plt.legend(loc='upper left')
# plt.savefig('coordinatexacc.PNG')

TESTING

In [31]:
import numpy as np
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, r2_score
# from keras.backend import clear_session
import pandas as pd

In [ ]:
yhat = model.predict(testImages)
y_true = [testLabels,testRotations,testCoordinates]

In [ ]:
#label,rotation,(x,y)
accuracy_score(np.argmax(y_true[0],axis=1),np.argmax(yhat[0],axis=1)),accuracy_score(np.argmax(y_true[1],axis=1),np.argmax(yhat[1],axis=1)),r2_score(y_true[2],yhat[2])

SINGLE TESTING

In [35]:

RA = df['RA (deg)'].tolist()[:37]
DE = df['DE (deg)'].tolist()[:37]

Magnitude = df['Magnitude'].tolist()[:37]
rotations = [0,120,144,168,192,216,24,240,264,288,312,336,48,72,96]

In [ ]:
image = cv2.imread('test_images/144trans150_0-Copy1.jpg')
image_show = np.array(image, dtype='float32')
image_show_crop = image_show[500:1700, 600:2300]
height, width = image_show.shape[:2]
image_input = cv2.resize(image_show, (256,256))
image_input_crop = image_input[58:166,51:174]
# plt.imshow(image_show[500:1700, 600:2300])
plt.imshow(image_input)

In [ ]:
y = model.predict(np.expand_dims(image_input,0))
label = y[0].argmax()
rotation = y[1].argmax()
co_x = y[2][0][0]
co_y = y[2][0][1]
cv2.circle(image_show, (int(np.around(co_x*width+width/2)),int(np.around(co_y*height+height/2))), 190, (255,255,255), thickness=2)
cv2.putText(image_show, 'RA=' + str(RA[label]) + '˚ DE=' + str(DE[label]) + '˚ ROLL=' + str(rotations[rotation]) + '˚', (100,100), cv2.FONT_HERSHEY_TRIPLEX, 4, (255,255,255), thickness=2)
plt.imshow(image_show)
print(RA[label],DE[label],ID[label],rotations[rotation],label)

CONVERTING TO TFLITE

In [61]:
import tensorflow as tf

In [ ]:
TF_LITE_MODEL_FILE_NAME = "model3.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = tf_lite_converter.convert()
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

In [ ]:
TF_LITE_MODEL_FILE_NAME = "model2.tflite"
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.resize_tensor_input(input_details[0]['index'], (2720, 256, 256, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (2720, 3))
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], testImages)
interpreter.invoke()
label = interpreter.get_tensor(output_details[2]['index'])
rotation = interpreter.get_tensor(output_details[0]['index'])
regression = interpreter.get_tensor(output_details[1]['index'])